In [ ]:
import sys
sys.path.append('./src/')
import numpy as np
from scipy import interpolate
from jcamp import JCAMP_reader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from rdkit import Chem
from scipy import stats

### IR spectrum reader from Jcamp file


In [ ]:

THRESHOLD_IR = 0.93
THRESHOLD_NMR = 0.02
curve_begin_idx = 600
curve_correction_factor = 0.5

class SpectraCarrier():
    def __init__(self, path, params=False):
        self.data_dic = JCAMP_reader(path)
        #print(self.data_dic)
        self.x = self.data_dic['x']
        self.y = self.data_dic['y']
        self.xfactor = self.data_dic['xfactor']
        self.yfactor = self.data_dic['yfactor']
        #self.molform = self.data_dic['molform']
        self.deltax = self.data_dic['deltax']
        self.firstx = self.data_dic['firstx']
        self.lastx = self.data_dic['lastx']
        self.firsty = self.data_dic['firsty']
        #self.maxx = self.data_dic['maxx']
        #self.minx = self.data_dic['minx']
        self.maxy = self.data_dic['maxy']
        self.miny = self.data_dic['miny']
    

def order(x_i, y_i):
    x_o = x_i
    y_o = y_i

    first_x = x_i[0]
    last_x = x_i[-1]
    if first_x > last_x:
        x_o = x_o[::-1]
        y_o = y_o[::-1]
    return x_o, y_o


def concat_boundary(x_i, y_i):
    x_head = np.array([0.0])
    x_tail = np.array([4000.0])
    x_o = np.concatenate([x_head, x_i, x_tail])

    y_head = np.array([y_i[0]])
    y_tail = np.array([y_i[-1]])
    y_o = np.concatenate([y_head, y_i, y_tail])
    return x_o, y_o


def sampling(x_i, y_i):
    f = interpolate.interp1d(x_i, y_i)
    x_o = np.linspace(0, 3999, 4000, endpoint=True)
    y_o = f(x_o)
    return x_o[curve_begin_idx:], y_o[curve_begin_idx:]


def normalize(x_i, y_i):
    max_y = np.max(y_i)
    min_y = np.min(y_i)
    height = max_y - min_y

    x_o = x_i
    y_o = (y_i - min_y) / height
    return x_o, y_o


def to_absorption(x_i, y_i):
    x_o = x_i
    y_o = -np.log10(y_i)
    
    return x_o, y_o


def chop(x_i, y_i):
    x_o = x_i[0:]
    y_o = y_i[0:]
    
    return x_o, y_o


def read_spectrum(path, to_absorption):
    sc = SpectraCarrier(path)

    # original
    x_orig, y_orig = order(sc.x, sc.y)

    # concatenate
    x_basis, y_basis = concat_boundary(x_orig, y_orig)

    #interpolate
    x_new, y_new = sampling(x_basis, y_basis)
    y_new = y_new ** curve_correction_factor

    #absorption
    if to_absorption :
        x_new_norm_abs, y_new_norm_abs = to_absorption(x_new, y_new)
    else :
        x_new_norm_abs, y_new_norm_abs = x_new, y_new

    #normalized
    x_new_norm, y_new_norm = normalize(x_new_norm_abs, y_new_norm_abs)

    # chomp
    x_nnac, y_nnac = chop(x_new_norm, y_new_norm)
    
    return x_nnac, y_nnac



### Code to read Chemotion dataset folder

In [ ]:

data_folder = 'data/exp/'
filepath = '{}meta_data.json'.format(data_folder)

columns = ['id', 'cano_smi', 'smiles', 'spectrum']
df = pd.DataFrame(columns=columns)

errors = []
cou = 0
with open(filepath, 'r') as f:
    data = json.load(f)

    for i in range(len(data)):
        smiles = data[i]['cano_smiles']
        id = data[i]['identifier']
        filepath = data_folder+id
        try:
            _, spec = read_spectrum(filepath, False)
        except (KeyError, Exception):
            errors.append(id)
            
        molfromsmi = Chem.MolFromSmiles(data[i]['cano_smiles'])
        if molfromsmi is None:
            errors.append(id)
            continue
        cano = Chem.MolToSmiles(Chem.MolFromSmiles(data[i]['cano_smiles']))
        row = [id, cano, smiles, spec] 
        df2 = pd.DataFrame(np.array(row).reshape(1,-1), columns= columns, index = [i])

        df = df.append(df2, ignore_index=True)
df.to_pickle('./chemIR_raw.pk')


### Code to read NIST dataset

In [ ]:
nist_data = 'data/nist/IR/cas-index.txt'
folderpath = 'data/nist/IR/Individual_Files/NIST/DX/'
casids = []

with open(nist_data, 'r') as f :
    d = f.readlines()
    for j in d:
        casids.append(j[0:15])

mol = np.load('data/nist/IR/cas-smiles.npy')

columns = ['id', 'cano_smi', 'smiles', 'spectrum']

df = pd.DataFrame(columns=columns)

for idx, i in enumerate(casids):
    filepath = folderpath+i.strip()+'.DX'
    errors = []
    try:
         _, spec = read_spectrum(filepath, False)
    except (KeyError, Exception) as e:
            errors.append(i)
            continue
    
    molfromsmi = Chem.MolFromSmiles(mol[idx])
    if molfromsmi is None:
        errors.append(id)
        continue
    cano = Chem.MolToSmiles(molfromsmi)

    # func = ifg.identify_functional_groups(molfromsmi)

    # rd_func = np.unique([i.type for i in func])

    # onehot = seq_onehot(rd_func, fg_lis)
    row = [idx, cano, mol[idx], spec] 
    # row.extend(onehot)
    df2 = pd.DataFrame(np.array(row).reshape(1,-1), columns= columns, index = [i])

    df = df.append(df2, ignore_index=True)

df.to_pickle('./nist.pk')
    

### Plot raw Chemotion dataset

In [ ]:
df_inhouse = np.load('./chemIR_raw.pk', allow_pickle= True)
plt.figure(figsize=(20,10))
for i in range(df_inhouse.shape[0]):
    plt.plot(df_inhouse.loc[i].spectrum)

### Z score based filtering for Chemotion dataset #

In [ ]:
arr = np.array([x for x in df_inhouse.spectrum]) 
arr = arr[:,3200:]
print(df_inhouse.shape)
xdf = df_inhouse[(np.abs(stats.zscore(arr)) < 2).all(axis=1)]
arr = np.array([x for x in xdf.spectrum]) 
arr = arr[:,1800:1900]
xdf = xdf[(np.abs(stats.zscore(arr)) < 2).all(axis=1)]
print(xdf.shape)
xdf = xdf.drop_duplicates(subset=['cano_smi'], keep='last')
xdf = xdf.reset_index(drop=True)
print(xdf.shape)
xdf.to_pickle('./chemIR_filtered.pk')

### Plot filtered Chemotion dataset

In [ ]:
plt.figure(figsize=(20,10))
for i in range(xdf.shape[0]):
    plt.plot(xdf.loc[i].spectrum)

### Code to label dataset with ground truth functional groups

In [ ]:
func_grp_smarts = {'alkane':'[CX4;H0,H1,H2,H4]',
                   'methyl':'[CH3]',
                   'alkene':'[CX3]=[CX3]',
                   'alkyne':'[CX2]#C',
                   'alcohols':'[#6][OX2H]',
                   'amines':'[NX3;H2,H1;!$(NC=O)]',
                   'nitriles':'[NX1]#[CX2]', 
                   'aromatics':'[$([cX3](:*):*),$([cX2+](:*):*)]',
                   'alkyl halides':'[#6][F,Cl,Br,I]', 
                   'esters':'[#6][CX3](=O)[OX2H0][#6]',
                   'ketones':'[#6][CX3](=O)[#6]',
                   'aldehydes':'[CX3H1](=O)[#6]', 
                   'carboxylic acids':'[CX3](=O)[OX2H1]',
                #    'ether': '[OD2]([#6])[#6]',
                   'ether': '[OD2]([#6;!$(C=O)])([#6;!$(C=O)])',
                   'acyl halides':'[CX3](=[OX1])[F,Cl,Br,I]',
                   'amides':'[NX3][CX3](=[OX1])[#6]',
                   'nitro':'[$([NX3](=O)=O),$([NX3+](=O)[O-])][!#8]'}




def identify_functional_groups(smiles):
    '''Identify the presence of functional groups present in molecule 
       denoted by smiles
    Args:
        root: (string) path to spectra data
        files: (list) jdx files present in root
        save_path: (string) path to store csv file
        bins: (np.array) used for standardizing
        is_mass: (bool) whether data being parsed is Mass or IR
    Returns:
        mol_func_groups: (list) contains binary values of functional groups presence
                          None if smiles to molecule conversion returns warning or error
    '''
    
    try:
        #Convert inchi to molecule
        mol = Chem.MolFromSmiles(smiles)#, treatWarningAsError=True)

        mol_func_grps = []

        #populate the list with binary values
        for _, func_struct in func_grp_structs.items():
            struct_matches = mol.GetSubstructMatches(func_struct)
            contains_func_grp = int(len(struct_matches)>0)
            mol_func_grps.append(contains_func_grp)
        return mol_func_grps
    except:

        return None

def save_target_to_csv(smiles_df, save_path, func_grp_structs):
    '''Save the target dataframe as csv to path
    Args:
        smiles_df: (pd.DataFrame) contains CAS and Inchi of molecules
        save_path: (string) path to store csv file
    Returns:
        None
    '''
    column_names = list(func_grp_structs.keys())
    column_names = ['id', 'cano_smi', 'smiles', 'spectrum'] + column_names
    target_df = pd.DataFrame(index = smiles_df.index, columns = column_names)

    #Iterate the rows, don't use df.apply since a list is being returned.
    for ind, (_, row) in enumerate(smiles_df.iterrows()):   

        labels = identify_functional_groups(row['cano_smi'])
        new_row = [ind, row['cano_smi'], row['smiles'], row['spectrum']] +labels
        target_df.iloc[ind, :] = new_row

    target_df.dropna(inplace = True)
    target_df.to_pickle(save_path)

smiles_df = np.load('./nist_epa.pk', allow_pickle=True)

func_grp_structs = {func_name : Chem.MolFromSmarts(func_smarts) for func_name, func_smarts in func_grp_smarts.items()}
save_target_to_csv(smiles_df, './nist_dataset.pk', func_grp_structs)


### Code to create Chemotion dataset subset

In [ ]:

nist = np.load('./nist_dataset.pk', allow_pickle=True)
inh = np.load('./chemIR_dataset.pk', allow_pickle=True)

nit_alk = inh[(inh['nitriles'] == 1) | (inh['alkyl halides'] == 1)]
nit_alk.to_pickle('./chemo_sub.pk')


### Code to concatenate different datasets

In [ ]:

df1 = np.load('./nist_dataset.pk', allow_pickle=True)
df2 = np.load('./chemo_sub.pk', allow_pickle=True)

pd.concat([df1,df2]).to_pickle('./mix_sub.pk')